# Project Part 2 Solutions

## (a)

In [13]:
import numpy as np
import pandas as pd
file=np.load("part2.npz")
beta_old=file['beta_old']
N=file['N']
Svc_0_PMF=file['Svc_0_PMF']
Lc=file['Lc']
Ic_0=file['Ic_0']
gamma=file['gamma']
L_observed=file['L_observed']
nb_nodes = Svc_0_PMF.shape[0]

In [14]:
def SIR(t, Z, N, betas, gamma):
    (S_vec, I_vec, R) = (Z[:16].reshape(4, 4), Z[16:20], Z[-1])
    I = I_vec.sum()
    dSdt = -betas*S_vec*I/N 
    dIdt = -dSdt.sum(axis=0)-gamma*I_vec
    dRdt = np.array([gamma*I])
    return np.concatenate((dSdt.flatten(), dIdt, dRdt))

In [34]:
from scipy.integrate import solve_ivp
from scipy.optimize import fmin
from sklearn.metrics import mean_squared_error

Svc_0_PMF /= Svc_0_PMF.sum(axis=(1, 2), keepdims=True)
Svc_0 = (N-Ic_0.sum(axis=1))[:, np.newaxis, np.newaxis]*Svc_0_PMF
Svc_0 = Svc_0.round().astype(int)
R0 = np.zeros((nb_nodes)).astype(int)

t = np.linspace(0, 29, 30)

S = np.zeros((nb_nodes, 4, 4, len(t)))
I = np.zeros((nb_nodes, 4, len(t)))
R = np.zeros((nb_nodes, len(t)))
L = np.zeros((nb_nodes, len(t)))

for N_node, node_index in zip(N, range(nb_nodes)):

    initial_conditions = np.concatenate(
        (Svc_0[node_index].flatten(), Ic_0[node_index], [R0[node_index]]))

    soln = solve_ivp(
        SIR, (t[0], t[9]), initial_conditions, args=(N_node, beta_old, gamma), t_eval=t[:10])

    S[node_index, :, :, :10] = soln.y[:16].reshape(4, 4, -1)
    I[node_index, :, :10] = soln.y[16:20]
    R[node_index, :10] = soln.y[20]
    L[node_index, :10] = Lc@soln.y[16:20]


alpha_predicted = np.zeros((nb_nodes))

# You will write a function that takes in a guess of α and returns the mean squared
# error between Lpredicted(t) and Lobserved(t) between days 10 and 29. You will find Lpredicted(t) the
# same way you did in Part 1 except the initial conditions will be the Sv,c, Ic and R values on
# day 9. Once this function is written, you can use scipy.optimize.fmin to find the α value that gives
# the minimum mean squared error. You can choose the starting point (second argument to fmin) as 0.5.

# this is the part i wrote:
def loss_func(alpha):
#     ###### WRITE YOUR FUNCTION HERE
    t = np.linspace(10, 29, 20)
    lPred = np.zeros((nb_nodes, len(t)))
    for N_node, node_index in zip(N, range(nb_nodes)):
        initial_conditions = np.concatenate((S[node_index, :, :, 9].flatten(), I[node_index, :, 9], R[node_index, 9]))
        # error here in initial conditions
        soln = solve_ivp(SIR, (t[0], t[-1]), initial_conditions, args=(N_node, alpha*beta, gamma), t_eval=t)
        lPred[node_index] = Lc@soln.y[16:20]
    mse=mean_squared_error(lTrue[9:29],lPred).tolist()
    return mse

for N_node, node_index in zip(N, range(nb_nodes)):
    alpha_predicted[node_index] = fmin(loss_func, 0.5, disp=False)
alpha_predicted

ValueError: all the input arrays must have same number of dimensions, but the array at index 0 has 1 dimension(s) and the array at index 2 has 0 dimension(s)

In [29]:
Svc_0

array([[[ 468,  487,  413,  285],
        [2176, 2108, 1665, 1071],
        [3632, 3278, 2410, 1444],
        [2176, 1829, 1253,  699]],

       [[ 830,  931,  845,  620],
        [4054, 4072, 3310, 2178],
        [6878, 6188, 4506, 2656],
        [4054, 3267, 2131, 1125]],

       [[2364, 2482, 2086, 1402],
        [4129, 3693, 2643, 1513],
        [2364, 1801, 1097,  535],
        [ 443,  288,  149,   62]],

       ...,

       [[  62,  102,  133,  138],
        [ 426,  579,  625,  535],
        [ 921, 1034,  921,  651],
        [ 625,  579,  426,  248]],

       [[1715, 5554, 8217, 5554],
        [4974, 3553, 1160,  173],
        [ 287,   45,    3,    0],
        [   0,    0,    0,    0]],

       [[  44,   44,   36,   24],
        [ 540,  521,  411,  265],
        [2441, 2280, 1741, 1088],
        [4035, 3649, 2699, 1633]]])

## (b)

In [ ]:
def g(money):
    alpha = 1/np.log2(2*10**(-3)*money+2)
    return alpha

def g_inv(alpha):
    money = (2**(1/alpha)-2)/(2*10**(-3))
    return money
    import matplotlib.pyplot as plt 

def scatter_plot(alpha_afterNPI):
    plt.figure()
    colors=(beta_old[np.newaxis,:,:]*Svc_0_PMF).mean(axis=(1,2))
    plt.scatter(alpha_predicted, alpha_afterNPI, s=N/500, c=colors, cmap='jet', alpha=0.5)
    plt.xlabel(r"$\alpha_{own}$")
    plt.ylabel(r"$\alpha_{after\ NPI}$")

def total_recovered_difference(alpha_afterNPI):
    
    ###### WRITE YOUR FUNCTION HERE
    
    return difference

### Policy #1

i.

ii.

### Policy #2

i. 

ii.

### Policy #3

i.

ii.

### Policy #4

i.

ii.

## (c)

### Policy 1

### Policy 2

### Policy 3

### Policy 4